# RLHF Canary - Quickstart

This notebook demonstrates how to use RLHF Canary to detect regressions in your RLHF/finetuning pipelines.

**What you'll learn:**
1. Run a DPO canary training job
2. Save metrics as a baseline
3. Compare new runs against baseline
4. Interpret regression reports

**Requirements:** GPU runtime (Runtime > Change runtime type > T4 GPU)

## 1. Setup

In [ ]:
# Install dependencies
!pip install -q transformers datasets accelerate trl peft bitsandbytes
!pip install -q pydantic click PyYAML

# Clone the repo
!git clone https://github.com/mmcmanus1/rlhf-canary.git
%cd rlhf-canary

In [ ]:
# Verify GPU is available
import torch
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

In [ ]:
# Show environment fingerprint
!python -m canary.cli env

## 2. Run Baseline Canary

In [ ]:
# Run DPO smoke test (takes ~5-10 min on T4)
!python -m canary.cli run configs/dpo_smoke.yaml -o ./canary_output

In [ ]:
# Find and display the metrics
import json
from pathlib import Path

metrics_files = list(Path('./canary_output').rglob('metrics.json'))
if metrics_files:
    baseline_path = metrics_files[0]
    print(f"Baseline metrics: {baseline_path}")
    
    with open(baseline_path) as f:
        metrics = json.load(f)
    
    print(f"\nRun ID: {metrics['run_id']}")
    print(f"Duration: {metrics['duration_seconds']:.1f}s")
    print(f"Step time (mean): {metrics['perf']['step_time']['mean']:.4f}s")
    print(f"Tokens/sec: {metrics['perf']['approx_tokens_per_sec']:.0f}")
    print(f"Peak memory: {metrics['perf']['max_mem_mb']:.0f}MB")
    print(f"NaN steps: {metrics['stability']['nan_steps']}")

In [ ]:
# Save as baseline
!mkdir -p baselines
!cp {baseline_path} baselines/dpo_baseline.json
print("Baseline saved to baselines/dpo_baseline.json")

## 3. Run Again and Compare

Now let's run another canary and compare it to our baseline.

In [ ]:
# Run another canary
!python -m canary.cli run configs/dpo_smoke.yaml -o ./canary_output

In [ ]:
# Find the new metrics file
metrics_files = sorted(Path('./canary_output').rglob('metrics.json'), key=lambda p: p.stat().st_mtime)
current_path = metrics_files[-1]  # Most recent
print(f"Current metrics: {current_path}")

In [ ]:
# Compare to baseline
!python -m canary.cli compare {current_path} baselines/dpo_baseline.json --threshold-tier smoke

## 4. Simulate a Regression

Let's intentionally create a slower config to see regression detection in action.

In [ ]:
# Create a "slower" config (smaller batch = more steps = slower)
slow_config = """
name: dpo_slow
description: Intentionally slow config for regression demo

model_name: EleutherAI/pythia-70m
use_peft: true
lora_r: 16
lora_alpha: 32
lora_dropout: 0.05

training_type: dpo
max_steps: 100
batch_size: 1  # Smaller batch = slower!
gradient_accumulation_steps: 8
learning_rate: 5.0e-5
max_length: 256
warmup_steps: 10

beta: 0.1
max_prompt_length: 64

dataset_name: Anthropic/hh-rlhf
dataset_split: train
dataset_size: 512
seed: 42

output_dir: ./canary_output
metrics_warmup_steps: 10
"""

with open('configs/dpo_slow.yaml', 'w') as f:
    f.write(slow_config)

print("Created slow config")

In [ ]:
# Run the slow config
!python -m canary.cli run configs/dpo_slow.yaml -o ./canary_output

In [ ]:
# Compare slow run to baseline - should show regression!
metrics_files = sorted(Path('./canary_output').rglob('metrics.json'), key=lambda p: p.stat().st_mtime)
slow_path = metrics_files[-1]

!python -m canary.cli compare {slow_path} baselines/dpo_baseline.json --threshold-tier smoke

## 5. Next Steps

- **PR Gating**: Add the GitHub Actions workflow to your repo
- **Nightly Tests**: Run longer `dpo_perf.yaml` tests overnight
- **Custom Thresholds**: Adjust thresholds for your use case
- **Root Cause Analysis**: Check the heuristics when regressions occur

See the [README](https://github.com/mmcmanus1/rlhf-canary) for more details.